**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

### Sequence Diagram

```mermaid

%%{
  init: {
    'theme': 'base',
    'themeVariables': {
      'primaryColor': '#ffffff',
      'primaryTextColor': '#000000',
      'primaryBorderColor': '#000000',
      'lineColor': '#000000',
      'secondaryColor': '#f4f4f4',
      'secondaryTextColor': '#000000',
      'tertiaryColor': '#d3d3d3',
      'tertiaryTextColor': '#000000',
      'background': '#ffffff',
      'actorBkg': '#B4B4B4',
      'actorTextColor': '#000000',
      'actorBorder': '#000000',
      'actorLineColor': '#000000',
      'signalColor': '#000000',
      'signalTextColor': '#000000',
      'activationBorderColor': '#000000',
      'activationBkgColor': '#d3d3d3',
      'sequenceNumberColor': '#000000',
      'noteBkgColor': '#F0F0F0',
      'noteTextColor': '#000000',
      'noteBorderColor': '#000000'
    }
  }
}%%

sequenceDiagram
    participant Platform as "Platform"
    participant Blockchain as "Iroha 1 Blockchain"

    Note over Platform, Blockchain: Deploy smart contract for account creation
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully
    
    Note over Platform, Blockchain: Account creation
    Platform->>Blockchain: Create Account
    Blockchain-->>Platform: Account Created Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm account creation
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
    
    
    Note over Platform, Blockchain: Deploy smart contract for details setting
    Platform->>Blockchain: Deploy Smart Contract
    Blockchain-->>Platform: Smart Contract Deployed Successfully

    Note over Platform, Blockchain: Set user details
    Platform->>Blockchain: Set User Details in Blockchain
    Blockchain-->>Platform: User Details Set Successfully

    Note over Platform, Blockchain: Queries the blockchain to confirm details setting
    Platform->>Blockchain: Query Account Details
    Blockchain-->>Platform: Query Response
```

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *



In [2]:
DOMAIN = "test"

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

In [3]:
from integration_helpers import get_engine_receipts_address
# - Deploy contract
create_account_contract_hash = create_account_contract()


2025-01-22 23:16:43.064 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_account_contract"
2025-01-22 23:16:43.070 | INFO     | iroha_helper:create_account_contract:41 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 23:16:43.071 | INFO     | iroha_helper:create_account_contract:41 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-22 23:16:45.488 | INFO     | iroha_helper:create_account_contract:41 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-22 23:16:45.525 | INFO     | iroha_helper:create_account_contract:41 - ('COMMITTED', 5, 0)
2025-01-22 23:16:45.527 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 23:16:45.547 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'760d265aee4878087bedc160f3cf5b3ef8bc3aedc102eb42e54e9e6ed67b036d'
2025-01-22 23:16:45.548 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: BD5A91148F7413A4E1B5

2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)


2025-01-22 23:16:45.602 | INFO     | __main__:<module>:13 - User Role: publisher
2025-01-22 23:16:45.604 | INFO     | __main__:<module>:24 - Creating account with name: optimistic_engelbart
2025-01-22 23:16:45.605 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 23:16:45.623 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'760d265aee4878087bedc160f3cf5b3ef8bc3aedc102eb42e54e9e6ed67b036d'
2025-01-22 23:16:45.624 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: BD5A91148F7413A4E1B522B14EC20F8CBAC94C24
2025-01-22 23:16:45.625 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-22 23:16:45.626 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"
2025-01-22 23:16:45.648 | INFO     | iroha_helper:create_user_account:83 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 23:16:45.65

2025-01-22 23:16:48.550 | INFO     | iroha_helper:create_user_account:83 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-22 23:16:48.579 | INFO     | iroha_helper:create_user_account:83 - ('COMMITTED', 5, 0)
2025-01-22 23:16:48.588 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 45
2025-01-22 23:16:48.589 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"
2025-01-22 23:16:48.591 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-22 23:16:48.609 | INFO     | iroha_helper:get_account:303 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 23:16:48.612 | INFO     | iroha_helper:get_account:303 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-22 23:16:51.599 | INFO     | iroha_helper:get_account:303 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-22 23:16:51.848 | INFO     | iroha_helper:get_account:303 - ('COMMITTED', 5, 0)
2025-01-22 23:16:51.851 | DEBUG    | integ

3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
#Query - GetAccountDetail

details = get_account_detail(f"{user_account_short_id}@{DOMAIN}")

2025-01-22 23:16:51.888 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account_detail"
2025-01-22 23:16:51.907 | INFO     | iroha_helper:get_account_detail:279 - User Account id = optimistic_engelbart@test, {}
2025-01-22 23:16:51.908 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account_detail"


4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting.

In [6]:
# - Deploys the contract
hash = create_detail_contract()

2025-01-22 23:16:51.921 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_detail_contract"
2025-01-22 23:16:51.940 | INFO     | iroha_helper:create_detail_contract:56 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 23:16:51.943 | INFO     | iroha_helper:create_detail_contract:56 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-22 23:16:54.869 | INFO     | iroha_helper:create_detail_contract:56 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-22 23:16:54.897 | INFO     | iroha_helper:create_detail_contract:56 - ('COMMITTED', 5, 0)
2025-01-22 23:16:54.899 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 23:16:54.919 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'5c8487c05800abb590a90924a0e2212db37def1a64a6ed8be9483646e840ffb2'
2025-01-22 23:16:54.920 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: E6924B053C1DA17BE4ABBFCAE

5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [7]:
address = integration_helpers.get_engine_receipts_address(hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-01-22 23:16:54.934 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-22 23:16:54.955 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'5c8487c05800abb590a90924a0e2212db37def1a64a6ed8be9483646e840ffb2'
2025-01-22 23:16:54.956 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: E6924B053C1DA17BE4ABBFCAE06C1B1DA7CE9D56
2025-01-22 23:16:54.957 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-22 23:16:54.959 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-01-22 23:16:54.961 | INFO     | iroha_helper:process_account:396 - Processing account: optimistic_engelbart@test
2025-01-22 23:16:54.962 | INFO     | iroha_helper:process_account:397 - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Optimistic Engelbart', 'foaf:mbox': 'optimistic_engelbart@email.com', 'foaf:or

6 - Queries the blockchain again to confirm the succesfull details setting.

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-01-22 23:16:57.988 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-22 23:16:58.006 | INFO     | iroha_helper:get_account:303 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-22 23:16:58.008 | INFO     | iroha_helper:get_account:303 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-22 23:17:00.974 | INFO     | iroha_helper:get_account:303 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-22 23:17:00.998 | INFO     | iroha_helper:get_account:303 - ('COMMITTED', 5, 0)
2025-01-22 23:17:01.000 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_account"
2025-01-22 23:17:01.002 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_result"
2025-01-22 23:17:01.023 | INFO     | integration_helpers:get_engine_receipts_result:139 - Result data: 00000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000002000000

In [ ]:
from typing import Optional
import json

def extract_user_json_ld_cid_from_data(data: dict) -> Optional[str]:
    try:
        # Clean the string to remove any trailing or extra characters
        json_data_str = data['json_data'].strip()  # This removes leading/trailing whitespaces or other invisible characters
        logger.debug(json_data_str)
        
        # Load the JSON data into a dictionary
        json_data = json.loads(json_data_str)
        logger.debug(f"Parsed JSON data:, {json_data}")
        # Debug print to check the structure of the parsed JSON
        # print("Parsed JSON data:", json_data)
        
        # Access the 'admin@test' key inside the parsed dictionary
        if 'admin@test' in json_data:
            admin_data = json_data['admin@test']
            logger.debug(f"Admin data:, {admin_data})  # Debug print to check the admin data
            
            # Check for 'user_json_ld_cid' inside the admin data
            if 'user_json_ld_cid' in admin_data:
                return admin_data['user_json_ld_cid']
            else:
                raise KeyError("'user_json_ld_cid' not found in the data")
        else:
            raise KeyError("'admin@test' not found in the parsed data")
            
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Invalid JSON data: {e}")
        return None

# data = {
#     "account_id": "friendly_montalcini@test",
#     "domain_id": "test",
#     "quorum": 1,
#     "json_data": "{\"admin@test\": {\"user_json_ld_cid\": \"QmXHvXvcZ4Yhs3kNKUtvsAUXTVtSFB8nS1JXjd5cdWoLeh\"}} "  # Trailing space added here
# }


print(details)

try:
    user_json_ld_cid = extract_user_json_ld_cid_from_data(details)
    if user_json_ld_cid:
        print("The value of 'user_json_ld_cid' is:", user_json_ld_cid)
    else:
        print("'user_json_ld_cid' not found in the data.")
except Exception as e:
    print(f"An error occurred: {e}")


                                                                                                                              {"account_id":"optimistic_engelbart@test","domain_id":"test","quorum":1,"json_data":"{\"admin@test\": {\"user_json_ld_cid\": \"QmZQVDLQ1yoRFh39wAXjNSs4dXTYwyP3qwDzKhcTZPnBNb\"}}"}             
An error occurred: string indices must be integers


7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [10]:
# Convert the JSON string to a Python dictionary
details_dict = json.loads(details)

# Now you can access the specific key like this
user_metadata_cid = details_dict["admin@test"]["user_json_ld_cid"]
logger.info(f"User Metadata CID: {user_metadata_cid}")

user_metadata = download_json_from_ipfs(user_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)